In [3]:
import subprocess
import glob
import os
import sys
import rasterio
from rasterio.plot import show
from pathlib import Path
import rasterio as rio
from tabulate import tabulate


base = '/home/hg_storage_one'

In [49]:
ls ~/images -R

/home/hg_storage_one/images:
20201001-20201031/  20201201-20201230/  20210220-20210316/  results/
20201101-20201130/  20210101-20210215/  20210317-20210416/

/home/hg_storage_one/images/20201001-20201031:
0000000000-0000000000.tif       0000000000-0000010496.tif
0000000000-0000000000_ndvi.tif  0000000000-0000010496_ndvi.tif

/home/hg_storage_one/images/20201101-20201130:
0000000000-0000000000.tif       0000000000-0000010496.tif
0000000000-0000000000_ndvi.tif  0000000000-0000010496_ndvi.tif

/home/hg_storage_one/images/20201201-20201230:
0000000000-0000000000.tif       0000000000-0000010496.tif
0000000000-0000000000_ndvi.tif  0000000000-0000010496_ndvi.tif

/home/hg_storage_one/images/20210101-20210215:
0000000000-0000000000.tif       0000000000-0000010496.tif
0000000000-0000000000_ndvi.tif  0000000000-0000010496_ndvi.tif

/home/hg_storage_one/images/20210220-20210316:
0000000000-0000000000.tif       0000000000-0000010496.tif
0000000000-0000000000_ndvi.tif  0000000000-0000010496_ndvi.ti

In [16]:
ls ~/data -R

/home/hg_storage_one/data:
classes.geojson    departamentos.dbf  departamentos.shx  verdad_campo.prj
clip.geojson       departamentos.prj  verdad_campo.cpg   verdad_campo.shp
departamentos.cpg  departamentos.shp  verdad_campo.dbf   verdad_campo.shx


In [62]:
!ogr2ogr -sql "SELECT * FROM departamentos WHERE nombre='PTE ROQUE SAENZ PENA'" -dialect sqlite {base}/data/departamentos_rsp.shp {base}/data/departamentos.shp

!ogr2ogr -sql "SELECT * FROM departamentos WHERE nombre='GENERAL VILLEGAS'" -dialect sqlite {base}/data/departamentos_villegas.shp {base}/data/departamentos.shp

!ogr2ogr -sql "SELECT * FROM departamentos WHERE nombre='GENERAL ROCA'" -dialect sqlite {base}/data/departamentos_roca.shp {base}/data/departamentos.shp

In [36]:
shapes = glob.glob('{base}/data/*.shp'.format(base= base))

for shp in shapes:
    layer = Path(shp).stem
    cmd = "ogrinfo -so {shp} {layer}".format(shp=shp, layer=layer)
    print(cmd)
    !{cmd}

ogrinfo -so /home/hg_storage_one/data/verdad_campo.shp verdad_campo
INFO: Open of `/home/hg_storage_one/data/verdad_campo.shp'
      using driver `ESRI Shapefile' successful.

Layer name: verdad_campo
Metadata:
  DBF_DATE_LAST_UPDATE=2021-04-24
Geometry: Point
Feature Count: 466
Extent: (-65.072036, -35.286969) - (-62.300375, -33.833709)
Layer SRS WKT:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["latitude",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["longitude",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
in1: String (6.0)
id: Integer64 (10.0)
cultivo: String (10.0)
ogrinfo -so /home/hg_storage_one/data/departamentos.shp departamentos
IN

## Calculamos indice NDVI

Bandas: 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'

NDVI: (NIR-RED)/(NIR+RED) -> (B8-B4)/(B8+B4) -> (imb7-imb3)/(imb7+imb3)

In [25]:
images = glob.glob('{base}/images/**/*.tif'.format(base= base))
for image in images:
    fname, ext = os.path.splitext(image)
    im_out_fname = '{im}_ndvi.tif'.format(im=fname)
    cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile;  otbcli_BandMath -il {im} -out {im_out} -exp "(im1b7-im1b3)/(im1b7+im1b3)"'""".format(im=image, im_out=im_out_fname)
    print(cmd)
    #subprocess.run(cmd, shell=True)

bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile;  otbcli_BandMath -il /home/hg_storage_one/images/20201101-20201130/0000000000-0000000000_ndvi.tif -out /home/hg_storage_one/images/20201101-20201130/0000000000-0000000000_ndvi_ndvi.tif -exp "(im1b7-im1b3)/(im1b7+im1b3)"'
bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile;  otbcli_BandMath -il /home/hg_storage_one/images/20201101-20201130/0000000000-0000010496_ndvi.tif -out /home/hg_storage_one/images/20201101-20201130/0000000000-0000010496_ndvi_ndvi.tif -exp "(im1b7-im1b3)/(im1b7+im1b3)"'
bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile;  otbcli_BandMath -il /home/hg_storage_one/images/20201101-20201130/0000000000-0000010496.tif -out /home/hg_storage_one/images/20201101-20201130/0000000000-0000010496_ndvi.tif -exp "(im1b7-im1b3)/(im1b7+im1b3)"'
bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile;  otbcli_BandMath -il /home/hg_storage_one/images/20201101-20201130/0000000000-0000000000.tif -out /home/hg_storage_one/images/20201101-2020

In [2]:
images = glob.glob('{base}/images/**/*_ndvi.tif'.format(base= base))

for image in images:
    print(image)
    #img = rasterio.open(image)
    #show(img)


/home/hg_storage_one/images/20201101-20201130/0000000000-0000000000_ndvi.tif
/home/hg_storage_one/images/20201101-20201130/0000000000-0000010496_ndvi.tif
/home/hg_storage_one/images/results/0000000000-0000000000_ndvi.tif
/home/hg_storage_one/images/results/0000000000-0000010496_ndvi.tif
/home/hg_storage_one/images/20201201-20201230/0000000000-0000000000_ndvi.tif
/home/hg_storage_one/images/20201201-20201230/0000000000-0000010496_ndvi.tif
/home/hg_storage_one/images/20210317-20210416/0000000000-0000000000_ndvi.tif
/home/hg_storage_one/images/20210317-20210416/0000000000-0000010496_ndvi.tif
/home/hg_storage_one/images/20210220-20210316/0000000000-0000000000_ndvi.tif
/home/hg_storage_one/images/20210220-20210316/0000000000-0000010496_ndvi.tif
/home/hg_storage_one/images/20201001-20201031/0000000000-0000000000_ndvi.tif
/home/hg_storage_one/images/20201001-20201031/0000000000-0000010496_ndvi.tif
/home/hg_storage_one/images/20210101-20210215/0000000000-0000000000_ndvi.tif
/home/hg_storage_on

## Juntamos las imagenes NDVI por tile

In [5]:
tiles = ['0000000000-0000000000', '0000000000-0000010496']

for tile in tiles:
    images_ndvi = sorted(glob.glob('{base}/images/**/{tile}_ndvi.tif'.format(tile=tile, base=base )))
    images_ndvi_otb = ' '.join(images_ndvi)
    im_out_fname = '{base}/images/results/{tile}_ndvi.tif'.format(tile=tile, base=base)
    cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile;  otbcli_ConcatenateImages -il {im} -out {im_out}'""".format(im=images_ndvi_otb, im_out=im_out_fname)
    #print(cmd)
    subprocess.run(cmd, shell=True)

## Clasificador por píxel

### Filenames

In [ ]:
image = '{base}/images/results/0000000000-0000010496_ndvi.tif'.format(base=base)
image2 = '{base}/images/results/0000000000-0000000000_ndvi.tif'.format(base=base)
classes_stat_filename = '{base}/results/classes_stat.xml'.format(base=base)
rates_filename = '{base}/results/rates.csv'.format(base=base)
samples_db = '{base}/results/samples.sqlite'.format(base=base)
image_stats_filename = '{base}/results/images_stats.xml'.format(base=base)
model_filename = '{base}/results/model.txt'.format(base=base)
cm_filename = '{base}/results/confusion_matrix_RF.csv'.format(base=base)
tile_name = Path(image).stem
predict_filename = '{base}/results/{tile_name}_predict.tif'.format(base=base, tile_name= tile_name)
tile_name2 = Path(image2).stem
predict_filename2 = '{base}/results/{tile_name2}_predict.tif'.format(base=base, tile_name2= tile_name2)
results_merge_filename =  '{base}/results/results_merge.tif'.format(base=base)
results_merge_filename_temp =  '{base}/results/results_merge_temp.tif'.format(base=base)
results_merge_soja_filename =  '{base}/results/results_merge_soja.tif'.format(base=base)
results_merge_maiz_filename =  '{base}/results/results_merge_maiz.tif'.format(base=base)
results_merge_girasol_filename =  '{base}/results/results_merge_girasol.tif'.format(base=base)
results_merge_soja_rsp_filename =  '{base}/results/results_merge_soja_rsp.tif'.format(base=base)
results_merge_maiz_rsp_filename =  '{base}/results/results_merge_maiz_rsp.tif'.format(base=base)
results_merge_soja_roca_filename =  '{base}/results/results_merge_soja_roca.tif'.format(base=base)
results_merge_maiz_roca_filename =  '{base}/results/results_merge_maiz_roca.tif'.format(base=base)
results_merge_soja_villegas_filename =  '{base}/results/results_merge_soja_villegas.tif'.format(base=base)
results_merge_maiz_villegas_filename =  '{base}/results/results_merge_maiz_villegas.tif'.format(base=base)


### PolygonClassStatistics

Pasamos la imagen y el archivo vectorial con las clases y nos devuelve un conteo de píxeles por clase. 

La columna con las clases es **id**

In [5]:
cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_PolygonClassStatistics  -in {image} \
                        -vec {base}/data/verdad_campo.shp \
                        -field  id \
                        -out {out}'""".format(image= image, base=base, out= classes_stat_filename)
print(cmd)
#subprocess.run(cmd, shell=True)

bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_PolygonClassStatistics  -in /home/hg_storage_one/images/results/0000000000-0000010496_ndvi.tif                         -vec /home/hg_storage_one/data/verdad_campo.shp                         -field  id                         -out /home/hg_storage_one/results/classes_stat.xml'


In [13]:
!cat {classes_stat_filename}

<?xml version="1.0" ?>
<GeneralStatistics>
    <Statistic name="samplesPerClass">
        <StatisticMap key="1" value="107" />
        <StatisticMap key="10" value="2" />
        <StatisticMap key="2" value="79" />
        <StatisticMap key="20" value="2" />
        <StatisticMap key="3" value="60" />
        <StatisticMap key="4" value="34" />
        <StatisticMap key="5" value="9" />
    </Statistic>
    <Statistic name="samplesPerVector">
        <StatisticMap key="0" value="1" />
        <StatisticMap key="1" value="1" />
        <StatisticMap key="10" value="1" />
        <StatisticMap key="100" value="1" />
        <StatisticMap key="101" value="1" />
        <StatisticMap key="102" value="1" />
        <StatisticMap key="103" value="1" />
        <StatisticMap key="104" value="1" />
        <StatisticMap key="105" value="1" />
        <StatisticMap key="106" value="1" />
        <StatisticMap key="107" value="1" />
        <StatisticMap key="108" value="1" />
        <Statistic

### SampleSelection

Recibe la imagen, el vector de clases, el archivo de stats de clases, el nombre de la clase (“id”), la estrategia de muestreo (smallest hace undersampling). Devuelve el porcentaje de observaciones por clase y los píxeles que van a ser muestreados.

El archivo samples.sqlite puede ser subido a un depósito de Storage y descargado en nuestra computadora para ser abierto en QGIS como un archivo vectorial (son puntos). 

In [6]:
cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_SampleSelection -in {image} \
                       -vec {base}/data/verdad_campo.shp \
                       -instats {classes_stat_filename} \
                       -field id \
                       -strategy smallest \
                       -outrates {rates_filename} \
                       -out {samples_db}'""".format(image= image, base=base, classes_stat_filename= classes_stat_filename, rates_filename= rates_filename, samples_db=samples_db)
print(cmd)
#subprocess.run(cmd, shell=True)

bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_SampleSelection -in /home/hg_storage_one/images/results/0000000000-0000010496_ndvi.tif                        -vec /home/hg_storage_one/data/verdad_campo.shp                        -instats /home/hg_storage_one/results/classes_stat.xml                        -field id                        -strategy smallest                        -outrates /home/hg_storage_one/results/rates.csv                        -out /home/hg_storage_one/results/samples.sqlite'


In [41]:
!cat {rates_filename}

#className requiredSamples totalSamples rate
1	2	107	0.0186916
10	2	2	1
2	2	79	0.0253165
20	2	2	1
3	2	60	0.0333333
4	2	34	0.0588235
5	2	9	0.222222


### SampleExtraction



In [7]:
cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_SampleExtraction -in {image} \
                        -vec {samples_db} \
                        -outfield prefix \
                        -outfield.prefix.name band_ \
                        -field id'""".format(image=image, base=base, samples_db=samples_db)
print(cmd)
#subprocess.run(cmd, shell=True)

bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_SampleExtraction -in /home/hg_storage_one/images/results/0000000000-0000010496_ndvi.tif                         -vec /home/hg_storage_one/results/samples.sqlite                         -outfield prefix                         -outfield.prefix.name band_                         -field id'


### ComputeImageStatistics

In [8]:
cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_ComputeImagesStatistics -il  {image} \
                               -out {image_stats_filename}'""".format(image=image, image_stats_filename=image_stats_filename)

print(cmd)
#subprocess.run(cmd, shell=True)

bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_ComputeImagesStatistics -il  /home/hg_storage_one/images/results/0000000000-0000010496_ndvi.tif                                -out /home/hg_storage_one/results/images_stats.xml'


### TrainVectorClassifier

In [9]:
cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_TrainVectorClassifier -io.vd {samples_db} \
			     -io.stats {image_stats_filename} \
                             -cfield id \
                             -classifier rf \
                             -classifier.rf.max 5 \
                             -classifier.rf.nbtrees 150 \
                             -io.out {model_filename} \
                             -io.confmatout {cm_filename} \
                             -feat band_0 band_1 band_2 band_3 band_4 band_5'""".format(samples_db=samples_db, image_stats_filename=image_stats_filename, model_filename=model_filename,
                                                                                       cm_filename= cm_filename)
print(cmd)
#subprocess.run(cmd, shell=True)

bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_TrainVectorClassifier -io.vd /home/hg_storage_one/results/samples.sqlite 			     -io.stats /home/hg_storage_one/results/images_stats.xml                              -cfield id                              -classifier rf                              -classifier.rf.max 5                              -classifier.rf.nbtrees 150                              -io.out /home/hg_storage_one/results/model.txt                              -io.confmatout /home/hg_storage_one/results/confusion_matrix_RF.csv                              -feat band_0 band_1 band_2 band_3 band_4 band_5'


### ImageClassifier

In [10]:
cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_ImageClassifier -in {image}  \
                       -imstat {image_stats_filename} \
                       -model {model_filename} \
                       -out {predict_filename}'""".format(image=image, image_stats_filename=image_stats_filename, model_filename=model_filename,
                                                                                       predict_filename= predict_filename)
print(cmd)
subprocess.run(cmd, shell=True)

bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_ImageClassifier -in /home/hg_storage_one/images/results/0000000000-0000010496_ndvi.tif                         -imstat /home/hg_storage_one/results/images_stats.xml                        -model /home/hg_storage_one/results/model.txt                        -out /home/hg_storage_one/results/0000000000-0000010496_ndvi_predict.tif'


CompletedProcess(args="bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_ImageClassifier -in /home/hg_storage_one/images/results/0000000000-0000010496_ndvi.tif                         -imstat /home/hg_storage_one/results/images_stats.xml                        -model /home/hg_storage_one/results/model.txt                        -out /home/hg_storage_one/results/0000000000-0000010496_ndvi_predict.tif'", returncode=0)

### ImageClassifier

In [11]:
cmd = """bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_ImageClassifier -in {image2}  \
                       -imstat {image_stats_filename} \
                       -model {model_filename} \
                       -out {predict_filename2}'""".format(image2=image2, image_stats_filename=image_stats_filename, model_filename=model_filename,
                                                                                       predict_filename2= predict_filename2)
print(cmd)
subprocess.run(cmd, shell=True)

bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_ImageClassifier -in /home/hg_storage_one/images/results/0000000000-0000000000_ndvi.tif                         -imstat /home/hg_storage_one/results/images_stats.xml                        -model /home/hg_storage_one/results/model.txt                        -out /home/hg_storage_one/results/0000000000-0000000000_ndvi_predict.tif'


CompletedProcess(args="bash -c 'source ~/OTB-7.2.0-Linux64/otbenv.profile; otbcli_ImageClassifier -in /home/hg_storage_one/images/results/0000000000-0000000000_ndvi.tif                         -imstat /home/hg_storage_one/results/images_stats.xml                        -model /home/hg_storage_one/results/model.txt                        -out /home/hg_storage_one/results/0000000000-0000000000_ndvi_predict.tif'", returncode=0)

### Unir resultados

In [12]:
!gdal_merge.py -ot UInt32 -o {results_merge_filename} {predict_filename} {predict_filename2}

#!gdal_translate -ot UInt32 {results_merge_filename_temp} {results_merge_filename}


### Armo raster por cultivo

In [13]:
!gdal_calc.py \
-A {results_merge_filename} \
--A_band=1 \
--calc="(((A==1) | (A==4))*1)+((A!=1) & (A!=4))*0" \
--outfile {results_merge_soja_filename}


In [14]:
!gdal_calc.py \
-A {results_merge_filename} \
--A_band=1 \
--calc="(((A==2) | (A==3))*1)+((A!=2) & (A!=3))*0" \
--outfile {results_merge_maiz_filename}

In [15]:
!gdal_calc.py \
-A {results_merge_filename} \
--A_band=1 \
--calc="(A==5)*1+(A!=5)*0" \
--outfile {results_merge_girasol_filename}

### Recortamos por cultivo y departamento

In [16]:
#ROQUE SAENZ PEÑA
!gdalwarp -cutline {base}/data/departamentos_rsp.shp -crop_to_cutline  {results_merge_soja_filename} {results_merge_soja_rsp_filename}
!gdalwarp -cutline {base}/data/departamentos_rsp.shp -crop_to_cutline  {results_merge_maiz_filename} {results_merge_maiz_rsp_filename}

#GENERAL ROCA
!gdalwarp -cutline {base}/data/departamentos_roca.shp -crop_to_cutline  {results_merge_soja_filename} {results_merge_soja_roca_filename}
!gdalwarp -cutline {base}/data/departamentos_roca.shp -crop_to_cutline  {results_merge_maiz_filename} {results_merge_maiz_roca_filename}

#GENERAL VILLEGAS
!gdalwarp -cutline {base}/data/departamentos_villegas.shp -crop_to_cutline {results_merge_soja_filename} {results_merge_soja_villegas_filename}
!gdalwarp -cutline {base}/data/departamentos_villegas.shp -crop_to_cutline  {results_merge_maiz_filename} {results_merge_maiz_villegas_filename}

### Resultados

In [19]:
def sum_pixels(file):
    raster = rio.open(file).read()
    raster_crop = raster[raster==1]
    sum_pixels = raster_crop.sum()
    return sum_pixels


def report(rsp_soja, rsp_maiz, roca_soja, roca_maiz, vil_soja, vil_maiz):
    rsp_soja = sum_pixels(rsp_soja)
    rsp_maiz = sum_pixels(rsp_maiz)
    roca_soja = sum_pixels(roca_soja)
    roca_maiz = sum_pixels(roca_maiz)
    villegas_soja = sum_pixels(vil_soja)
    villegas_maiz = sum_pixels(vil_maiz)
    
    pixel_to_ha = 0.04

    final_results = [['Departamento', 'Pixeles Maíz', 'Pixeles Soja', 'Hectareas Maíz', 'Hectareas Soja'],
                     ['Roque Saenz Peña', rsp_maiz, rsp_soja, rsp_maiz*pixel_to_ha, rsp_soja*pixel_to_ha ],
                     ['General Roca', roca_maiz, roca_soja, roca_maiz*pixel_to_ha, roca_soja*pixel_to_ha],
                     ['General Villegas', villegas_maiz, villegas_soja, villegas_maiz*pixel_to_ha, villegas_soja*pixel_to_ha]]
    
    print(tabulate(final_results))

In [20]:
report(results_merge_soja_rsp_filename, 
       results_merge_maiz_rsp_filename, 
       results_merge_soja_roca_filename, 
       results_merge_maiz_roca_filename, 
       results_merge_soja_villegas_filename, 
       results_merge_maiz_villegas_filename)

----------------  ------------  ------------  ------------------  ------------------
Departamento      Pixeles Maíz  Pixeles Soja  Hectareas Maíz      Hectareas Soja
Roque Saenz Peña  2262190       13451018      90487.6             538040.72
General Roca      6550238       29020461      262009.52000000002  1160818.44
General Villegas  1050793       2991460       42031.72            119658.40000000001
----------------  ------------  ------------  ------------------  ------------------
